## Watson Final Project Data Prep

This notebook prepares data for insertion into VeloBall's multiclass classification model for informing expected value calculations for teams sending riders to grand tours, the biggest races in cycling.

In [1]:
# ===== Imports ===== #
import json
import pandas as pd

### Raw data import
Raw data comes from structured files containing data related to grand tour (and smaller 1-week stage race) results and time trial (TT) results of riders. Structured data loaded here comes from unstructured data scraped from procyclingstats.com, an online database which tracks men's and women's professional cycling results.

In [2]:
# ===== Load GC data ===== #
with open('../system-data/men-gc-1995-2023-eval.json') as f:
    gc_raw = json.load(f)
f.close()

# ===== Load TT data ===== #
with open('../system-data/men-itt-1995-2023-eval.json') as f:
    tt_raw = json.load(f)
f.close()

### Data cleaning

In [19]:
# ===== Select only GC data from years post-1998 and adjust keys in the data ===== #
# this cell will create the dictionary gc_data, which contains keys 
# "actual" and "top_active". The "actual" key contains a list (in order of finish) of the actual
# finishes of riders at that race. "top_active" is a list of the top 100 finishers of the
# race, in decreasing order of their GC Elo rating going in (Elo ratings explained in my report).

# init dict for cleaned GC data
gc_data = {}

for key in gc_raw:
    
    # if the key does not include one of the years which we are excluding, remove "-final-gc" from the
    # key and add to the new gc_data dict
    if ('1995' not in key and '1996' not in key and '1997' not in key and '1998' not in key):
        
        # remove unused field
        gc_raw[key]['predicted']
        
        # add to gc_data
        gc_data[key.replace('-final-gc', '')] = gc_raw[key]

gc_data['tour-de-france-2021']

{'actual': ['POGAČAR Tadej',
  'VINGEGAARD Jonas',
  'CARAPAZ Richard',
  "O'CONNOR Ben",
  'KELDERMAN Wilco',
  'MAS Enric',
  'LUTSENKO Alexey',
  'MARTIN Guillaume',
  'BILBAO Pello',
  'URÁN Rigoberto',
  'GAUDU David',
  'CATTANEO Mattia',
  'CHAVES Esteban',
  'MEINTJES Louis',
  'PARET-PEINTRE Aurélien',
  'POELS Wout',
  'TEUNS Dylan',
  'GUERREIRO Ruben',
  'VAN AERT Wout',
  'MOLLEMA Bauke',
  'HENAO Sergio',
  'BONNAMOUR Franck',
  'CASTROVIEJO Jonathan',
  'VALVERDE Alejandro',
  'HIGUITA Sergio',
  'IZAGIRRE Ion',
  'KONRAD Patrick',
  'QUINTANA Nairo',
  'MEURISSE Xandro',
  'ALAPHILIPPE Julian',
  'MOHORIČ Matej',
  'KUSS Sepp',
  'BUCHMANN Emanuel',
  'MAJKA Rafał',
  'PACHER Quentin',
  'ELISSONDE Kenny',
  'BERNARD Julien',
  'PORTE Richie',
  'STUYVEN Jasper',
  'MARTIN Dan',
  'THOMAS Geraint',
  'MADOUAS Valentin',
  'POWLESS Neilson',
  'FORMOLO Davide',
  'DONOVAN Mark',
  'RODRÍGUEZ Cristián',
  'LATOUR Pierre',
  'BAKELANTS Jan',
  'KÜNG Stefan',
  'POLITT Nils

In [20]:
# ===== Select only TT data which is not in the years 1995 or 1996 ===== #
# output dictionary is the same format as gc_data, except that it contains
# a "tt_kms" column, which is the sum of kms of time trialing included in
# the race corresponding to the current key

# init tt data dictionary
tt_data = {}

# temp dict, before TTs info from the same race are aggregated
tt_temp = {}
for key in tt_raw:
    
    # select TT data post-1996
    if ('1995' not in key and '1996' not in key):
        tt_temp[key] = tt_raw[key]

for key in tt_temp:
    
    # get the name of the race from the key
    split = key.split('-')
    race_key = split[0]
    
    # extract just the name of the race from the key (races may have multiple TTs associated with them,
    # which makes the keys more complicated and in need of simplifying)
    s = 1
    while not split[s].isdigit():
        race_key += f'-{split[s]}'
        s += 1
    race_key += f'-{split[s]}'
    
    if race_key not in tt_data:
        tt_data[race_key] = {'top_active': tt_temp[key]['top_active'], 'tt_kms': tt_temp[key]['length']}
    else:
        tt_data[race_key]['tt_kms'] += tt_temp[key]['length']

tt_data['tour-de-france-2021']

{'top_active': [['ROGLIČ Primož', 1639.5227991531572],
  ['KÜNG Stefan', 1607.2605272327728],
  ['THOMAS Geraint', 1595.3728388815232],
  ['VAN AERT Wout', 1589.9296770541598],
  ['BISSEGGER Stefan', 1580.2364244049932],
  ['POGAČAR Tadej', 1578.5905547238551],
  ['CAMPENAERTS Victor', 1574.097224365755],
  ['PORTE Richie', 1570.1007905512417],
  ['ALAPHILIPPE Julian', 1562.2407382167667],
  ['ASGREEN Kasper', 1562.0683523772966],
  ['KRAGH ANDERSEN Søren', 1557.1047840626718],
  ['CATTANEO Mattia', 1555.732750720491],
  ['MCNULTY Brandon', 1554.9506933469058],
  ['URÁN Rigoberto', 1551.5526288289282],
  ['KELDERMAN Wilco', 1542.098558999018],
  ['IZAGIRRE Ion', 1542.0781741394132],
  ['WALSCHEID Max', 1541.7930389671446],
  ['DE GENDT Thomas', 1540.867821792711],
  ['KWIATKOWSKI Michał', 1540.629976260537],
  ['LUTSENKO Alexey', 1539.4890924911938],
  ['KRUIJSWIJK Steven', 1536.6946720671403],
  ['CASTROVIEJO Jonathan', 1536.5926222941491],
  ['MARTIN Tony', 1531.1216034447107],
  ['B

In [22]:
def get_gt_results(raw_data, race_key, rider):
    """
    Given cleaned data, a race key, and a rider name, get counts for the number of
    grand tour wins, podiums, and top 10s that rider has achieved.
    """
    
    # init counts for wins, podiums, top10s
    wins = 0
    podiums = 0
    top10s = 0
    
    for key in raw_data:
        
        # we are counting consecutively through time up until the race in question,
        # so if key == race_key, then we break out of the loop because every key
        # after and including that of the race_key is after the race corresponding
        # to the race_key
        if key.replace('-final-gc', '') == race_key: 
            break
            
        # make sure only grand tour results are collected
        if 'italia' not in key and 'espana' not in key and 'france' not in key:
            continue
            
        # get the result of the race
        res = raw_data[key]['actual']
         
        if rider in res:
            
            # update result counts for the rider
            actual = res.index(rider)
            if actual == 0:
                wins += 1
            if actual > 0 and actual <= 2:
                podiums += 1
            if actual > 2 and actual <= 9:
                top10s += 1
    
    return wins, podiums, top10s

### Final data aggregation
Final sorting of data into the features which will be used for the VeloBall model. Features are discussed in more detail in the report, but here we track features where each rider and their profile entering a grand tour correspond to a sample. Features for each sample include features tracking rider GC Elo ratings, their GC Elo ratings relative to the other race entrants, TT Elo ratings, TT Elo ratings relative to the other entrants, grand tour history of the rider, and the number of kilometers of time trialing in the race. Further, the "actual" column encodes the ultimate result of the rider in the race. Labels 0-9 correspond to finishing places 1-10, and label 10 corresponds to a finish outside the top 10 of the grand tour. Identifying info about the rider and race of each sample is also included in the dataset, for now.

In [25]:
# "success position" allows us to encode each finishing posiiton up to the 
# success_position with a label equal to that finish minus 1, and for us to encode a finish 
# worse than the success_position simply as success_position. Example: if success_position is 10
# and a rider finishes 10th, then the rider's result is labeled 9. If success_position is 10
# and a rider finishes 11th or worse, their label is 10.
success_position = 10

# dictionary containing all the features
comb = {
    'race': [], 'rider': [], 'elo_ranking': [], 'elo_rating': [], 
    'tt_elo_ranking': [], 'tt_elo_rating': [], '1_diff': [], '3_diff': [], 
    '5_diff': [], '10_diff': [], 'tt_1_diff': [], 'tt_3_diff': [], 
    'tt_5_diff': [], 'tt_10_diff': [], 'tt_kms': [], 'grand_tour_wins': [], 
    'grand_tour_podiums': [], 'grand_tour_top10s': [], 'actual': []
}

for key in gc_data:
    
    # only include grand tour data
    if 'giro' not in key and 'vuelta' not in key and 'tour' not in key:
        continue
    
    # extract TT features
    try:
        tt_kms = tt_data[key.replace('-final-gc', '')]['tt_kms']
        tt_rating1 = tt_data[key.replace('-final-gc', '')]['top_active'][0][1]
        tt_rating3 = tt_data[key.replace('-final-gc', '')]['top_active'][2][1]
        tt_rating5 = tt_data[key.replace('-final-gc', '')]['top_active'][4][1]
        tt_rating10 = tt_data[key.replace('-final-gc', '')]['top_active'][9][1]
    except:
        tt_kms = 0
        tt_rating1 = 1500
        tt_rating3 = 1500
        tt_rating5 = 1500
        tt_rating10 = 1500
    
    # extract rider profile features
    for rank, t in enumerate(gc_data[key]['top_active']):
        
        rider = t[0]
        rating = t[1]
        
        try:
            tt_rank = len(tt_data[key.replace('-final-gc', '')]['top_active']) + 1
            tt_rating = 1500
            for idx, tup in enumerate(tt_data[key.replace('-final-gc', '')]['top_active']):
                if tup[0] == rider:
                    tt_rank = idx + 1
                    tt_rating = tup[1]
        except:
            tt_rank = 0
            tt_rating = 1500
    
        
        # encode rider finish, for labels
        actual_result = (
            gc_data[key]['actual'].index(rider) 
            if rider in gc_data[key]['actual'][0: success_position]
            else success_position
        )

        # get grand tour history of the rider
        wins, podiums, grand_tour_top10s = get_gt_results(gc_raw, key, rider)
        
        # fill in data for this sample
        comb['race'].append(key)
        comb['rider'].append(rider)
        comb['elo_ranking'].append(rank + 1)
        comb['elo_rating'].append(rating)
        comb['tt_elo_ranking'].append(tt_rank)
        comb['tt_elo_rating'].append(tt_rating)
        comb['1_diff'].append(gc_data[key]['top_active'][0][1] - rating)
        comb['3_diff'].append(gc_data[key]['top_active'][2][1] - rating)
        comb['5_diff'].append(gc_data[key]['top_active'][4][1] - rating)
        comb['10_diff'].append(gc_data[key]['top_active'][9][1] - rating)
        comb['tt_1_diff'].append(tt_rating1 - tt_rating)
        comb['tt_3_diff'].append(tt_rating3 - tt_rating)
        comb['tt_5_diff'].append(tt_rating5 - tt_rating)
        comb['tt_10_diff'].append(tt_rating10 - tt_rating)
        comb['tt_kms'].append(tt_kms)
        comb['grand_tour_wins'].append(wins)
        comb['grand_tour_podiums'].append(podiums)
        comb['grand_tour_top10s'].append(grand_tour_top10s)
        comb['actual'].append(actual_result)


df = pd.DataFrame(data = comb)
df.head(10)

,race,rider,elo_ranking,elo_rating,tt_elo_ranking,tt_elo_rating,1_diff,3_diff,5_diff,10_diff,tt_1_diff,tt_3_diff,tt_5_diff,tt_10_diff,tt_kms,grand_tour_wins,grand_tour_podiums,grand_tour_top10s,actual
0,giro-d-italia-2014,QUINTANA Nairo,1,1570.818289,12,1530.875226,0.000000,-25.099039,-43.810856,-57.803394,61.644675,14.438616,9.457752,1.313071,69.0,0,0,0,0
1,giro-d-italia-2014,POZZOVIVO Domenico,2,1546.975508,11,1531.921596,23.842781,-1.256257,-19.968075,-33.960613,60.598305,13.392246,8.411381,0.266701,69.0,0,0,0,4
2,giro-d-italia-2014,EVANS Cadel,3,1545.719250,4,1541.509749,25.099039,0.000000,-18.711817,-32.704355,51.010152,3.804094,-1.176771,-9.321452,69.0,0,0,0,7
3,giro-d-italia-2014,KIŠERLOVSKI Robert,4,1530.029632,38,1507.496594,40.788657,15.689619,-3.022198,-17.014737,85.023307,37.817248,32.836384,24.691703,69.0,0,0,0,9
4,giro-d-italia-2014,ARREDONDO Julián David,5,1527.007433,69,1499.567984,43.810856,18.711817,0.000000,-13.992538,92.951917,45.745859,40.764994,32.620313,69.0,0,0,0,10
5,giro-d-italia-2014,KELDERMAN Wilco,6,1521.333960,13,1529.818995,49.484328,24.385290,5.673473,-8.319065,62.700906,15.494847,10.513983,2.369302,69.0,0,0,0,6
6,giro-d-italia-2014,MORENO Daniel,7,1519.326341,46,1503.512021,51.491948,26.392909,7.681092,-6.311446,89.007880,41.801821,36.820957,28.676276,69.0,0,0,0,10
7,giro-d-italia-2014,GENIEZ Alexandre,8,1517.283415,27,1511.541592,53.534874,28.435835,9.724018,-4.268520,80.978309,33.772250,28.791386,20.646705,69.0,0,0,0,10
8,giro-d-italia-2014,CUNEGO Damiano,9,1513.150776,62,1500.101635,57.667513,32.568474,13.856657,-0.135881,92.418266,45.212208,40.231343,32.086662,69.0,0,0,0,10
9,giro-d-italia-2014,URÁN Rigoberto,10,1513.014895,9,1533.445569,57.803394,32.704355,13.992538,0.000000,59.074332,11.868274,6.887409,-1.257272,69.0,0,0,0,1
